In [ ]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

DuckDB supports Python UDFs— so if you'd like to use a function in SQL on a dataset, you can!

In [ ]:
import duckdb
from duckdb.typing import *
from faker import Faker


def random_emoji():
    fake = Faker()
    fake.add_provider("emoji")
    return fake.emoji()


conn.create_function("random_emoji", random_emoji, [], "VARCHAR", side_effects=True)

In [ ]:
%%sql
SELECT
    fullname,
    random_emoji() as 👋
FROM nps_public_data.parks
LIMIT 10;
    

In [ ]:
import pyarrow.compute as pc


def swap_case(x):
    # Swap the case of the 'column' using utf8_swapcase and return the result
    return pc.utf8_swapcase(x)


# To register the function, we must define it's type to be 'arrow'
conn.create_function("swap_case", swap_case, ["VARCHAR"], "VARCHAR", type="arrow")

In [ ]:
%%sql
SELECT
    swap_case(fullname) as pARK_nAME
FROM nps_public_data.parks
LIMIT 10